<br><br><center><h1 style="font-size:3em">Praxisprojekt Datenbanken und SQL</h1></center>
<center><h3>Projekt 3 - ToDo Liste - Anton, Lisa, Maik</b></center>

### Standarteinstellungen

In [2]:
# Importe
import pandas as pd
import psycopg                        
import yaml                            

import sqlalchemy as sa 
import datetime as dt  
from sqlalchemy import text


In [3]:
# Standart-config-Datei laden
with open('..\\config.yaml', 'r') as file:
    config = yaml.load(file, Loader=yaml.BaseLoader)

# Connection-String
connection_string = f'postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/' 

# Verbindungen aufbauen
sa_eng = sa.create_engine(connection_string, 
                          isolation_level='AUTOCOMMIT')


In [5]:
# Datenbank todo erstellen
with sa_eng.connect() as con:            
    con.execute(text('''CREATE DATABASE todo'''))

In [6]:
# Neue Verbindung zur Datenbank erstellen

# alte Verbindung trennen
sa_eng.dispose()

# Verbindung zur todo-Datenbank erstellen

con_string_todo = f'postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo'

# Verbindungen aufbauen
sa_eng = sa.create_engine(con_string_todo, 
                          isolation_level='AUTOCOMMIT')



### Tabellen erstellen und definieren

In [9]:
# TABELLE - PROJEKTE
# projekt_id serial PRIMARY KEY,
# projekt_name text NOT NULL,
# beschreibung text,
# startdatum date NOT NULL
# enddatum date NOT NULL

with sa_eng.connect() as con:            
    con.execute(text('''
                    CREATE TABLE projekte(
                        projekt_id SERIAL PRIMARY KEY,
                        projekt_name TEXT NOT NULL,
                        beschreibung TEXT,
                        startdatum DATE NOT NULL,
                        enddatum DATE NOT NULL);
                     '''))

# TABELLE - TEAM
# mitarbeiter_id PRIMARY KEY
# vorname
# nachname
# rolle (projektleitung, entwicklung, design,...)
# email

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE team(
            mitarbeiter_id SERIAL PRIMARY KEY,
            vorname TEXT NOT NULL,
            nachname TEXT NOT NULL,
            rolle TEXT,
            email TEXT NOT NULL
        );
    '''))

# TABELLE - AUFGABEN
# aufgaben_id int PRIMARY KEY
# aufgaben_name
# projekt_id int FOREIGN KEY
# aufgaben_beschreibung text
# mitarbeiter_id FOREIGN KEY
# status text

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE aufgaben(
            aufgaben_id SERIAL PRIMARY KEY,
            aufgaben_name TEXT NOT NULL,
            projekt_id INT,
            aufgaben_beschreibung TEXT,
            mitarbeiter_id INT,
            status TEXT,
            FOREIGN KEY (projekt_id) REFERENCES projekte(projekt_id),
            FOREIGN KEY (mitarbeiter_id) REFERENCES team(mitarbeiter_id)
        );
    '''))

### Funktionen und Trigger